In [2]:
from keras.utils import plot_model
import prophet as ph

Using TensorFlow backend.


In [3]:
plot_model(ph.prophet1(), to_file='ph1.png', show_shapes=True)
plot_model(ph.prophet2(), to_file='ph2.png', show_shapes=True)

In [137]:
import requests
def bitfinex(tf, pair, limit=1000):
    url = 'https://api.bitfinex.com/v2/candles/trade:{}:t{}/hist?limit={}'.format(tf, pair, limit)
    request = requests.get(url)
    response = request.json()
    points = []
    if isinstance(response, list) and response[0] != 'error':
        count = 0 
        for row in response:  
            json_body = {
                        "open": float(row[1]),
                        "close": float(row[2]),
                        "high": float(row[3]),
                        "low": float(row[4]),
                        "volume": float(row[5]),
                        }
            points.append(json_body)
    
    return points

In [138]:
data = bitfinex('1h', 'BTCUSD')

In [139]:
import pandas as pd
import numpy as np

In [140]:
df = pd.DataFrame(data)

In [141]:
df.shape

(1000, 5)

In [33]:
def make_samples(data, length=40):
    samples = [data[i:i+length+1] for i in range(len(data)) \
               if len(data[i:i+length+1])==length+1]
    return samples


In [142]:
samples = make_samples(data, 60)

In [143]:
samples_close = [np.array([s['close'] for s in x]) for x in samples]

In [144]:
sc = [(x-np.min(x))/(np.max(x)-np.min(x)) for x in samples_close]

In [174]:
X = np.array([s[:-1].reshape(60,1) for s in sc])
Y = np.array([s[-1] for s in sc])

In [175]:
from sklearn.model_selection import train_test_split

In [176]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [179]:
model = ph.prophet1(ts_input_shape=60, lstm_size=32)

In [180]:
results = model.fit(
 X_train, y_train,
 epochs= 30,
 batch_size = 200,
 validation_data = (X_test, y_test)
)

Train on 752 samples, validate on 188 samples
Epoch 1/30
752/752 [==============================] - 2s 3ms/step - loss: 0.3132 - val_loss: 0.1367
Epoch 2/30
752/752 [==============================] - 0s 376us/step - loss: 0.1036 - val_loss: 0.0688
Epoch 3/30
752/752 [==============================] - 0s 382us/step - loss: 0.0664 - val_loss: 0.0559
Epoch 4/30
752/752 [==============================] - 0s 390us/step - loss: 0.0598 - val_loss: 0.0529
Epoch 5/30
752/752 [==============================] - 0s 431us/step - loss: 0.0577 - val_loss: 0.0511
Epoch 6/30
752/752 [==============================] - 0s 390us/step - loss: 0.0561 - val_loss: 0.0496
Epoch 7/30
752/752 [==============================] - 0s 387us/step - loss: 0.0546 - val_loss: 0.0481
Epoch 8/30
752/752 [==============================] - 0s 411us/step - loss: 0.0532 - val_loss: 0.0469
Epoch 9/30
752/752 [==============================] - 0s 401us/step - loss: 0.0518 - val_loss: 0.0456
Epoch 10/30
752/752 [=================

In [188]:
p = model.predict(np.array(X[-61:]))

In [189]:
r = Y[-61:]

In [ ]:
plt.plot([x[0] for x in p])

In [167]:
import matplotlib.pyplot as plt